#### Comparar grafos con los de SolidityAbstractor
Compara los grafos resultantes de correr este programa con VeriSol con los resultantes de SolidityAbstractor, bajo las mismas condiciones (`txBound` y `timeout`).
Es sólo un sanity check para ver que no haya ningún bug en mi programa y que los grafos sean iguales.

In [1]:
import os
import json5 as json
import pygraphviz as pgv

In [2]:
class MyGraph:
    def __init__(self, graph):
        self.graph = pgv.AGraph(graph)
        self.nodes = set(self.graph.nodes())
        self.edges = self.get_edges_with_labels()
        self.labels_of_nodes = self.get_labels_of_nodes()

    def get_edges_with_labels(self):
        return set(map(lambda x: (x, x.attr["label"].strip()), self.graph.edges()))
    
    def get_labels_of_nodes(self):
        return set(map(lambda x: x.attr["label"].strip(), self.graph.nodes()))

    def equals(self, other_graph):
        return self.edges == other_graph.edges and self.labels_of_nodes == other_graph.labels_of_nodes


In [3]:
VERISOL_GRAPHS_DIR = "../results/verisol_output"
contracts = os.listdir(VERISOL_GRAPHS_DIR)
dirs = [os.path.join(VERISOL_GRAPHS_DIR, contract) for contract in contracts]
dirs = [os.path.join(d, subdir) for d in dirs for subdir in ["epa/8", "states/8"]]

In [4]:
full_dirs = []

for d in dirs:
    if os.path.exists(os.path.join(d, "graph")):
        d = os.path.join(d, "graph")
    else:
        d = os.path.join(d, "reduce_combinations/graph")
    
    full_dirs.append(d)

In [5]:
print("-- Before --")
print(dirs[0])
print(dirs[1])
print()
print("-- After --")
print(full_dirs[0])
print(full_dirs[1])

-- Before --
../results/verisol_output/HelloBlockchain_fixed/epa/8
../results/verisol_output/HelloBlockchain_fixed/states/8

-- After --
../results/verisol_output/HelloBlockchain_fixed/epa/8/graph
../results/verisol_output/HelloBlockchain_fixed/states/8/graph


In [6]:
def fix_filename(filename):
    filename = filename.replace("_fixed", "")
    filename = filename.replace("CrowdfundingT_BaseBalanceFix", "CrowdfundingBase")
    filename = filename.replace("CrowdfundingT_BaseBalance", "CrowdfundingBase")
    filename = filename.replace("RefundEscrowWithdraw", "RefundEscrow")
    return filename

filenames = [f"{fix_filename(f)}_{mode}" for f in contracts for mode in ["Mode.epa", "Mode.states"]]

In [7]:
# Link / join each filename with its corresponding directory (they are already in order) from full_dirs
# So that results in the path of each graph file
graphs = list(map(lambda x: os.path.join(*x), zip(full_dirs, filenames)))

In [8]:
# Check if all files in graphs are valid files and return false otherwise
def check_files(graphs):
    no_estan = []
    for graph in graphs:
        if not os.path.isfile(graph):
            print(f"File {graph} does not exist")
            no_estan.append(graph)
    return no_estan

check_files(graphs) # Hay algunos grafos que no corrimos en modo states por ejemplo, así que está bien que no exsitan.

File ../results/verisol_output/Auction/states/8/reduce_combinations/graph/Auction_Mode.states does not exist
File ../results/verisol_output/AuctionWithdraw/epa/8/reduce_combinations/graph/AuctionWithdraw_Mode.epa does not exist
File ../results/verisol_output/AuctionWithdraw/states/8/reduce_combinations/graph/AuctionWithdraw_Mode.states does not exist
File ../results/verisol_output/SimpleAuctionHB/epa/8/reduce_combinations/graph/SimpleAuctionHB_Mode.epa does not exist
File ../results/verisol_output/SimpleAuctionHB/states/8/reduce_combinations/graph/SimpleAuctionHB_Mode.states does not exist
File ../results/verisol_output/CrowdfundingT_BaseBalance/states/8/reduce_combinations/graph/CrowdfundingBase_Mode.states does not exist
File ../results/verisol_output/SimpleAuctionEnded/epa/8/reduce_combinations/graph/SimpleAuctionEnded_Mode.epa does not exist
File ../results/verisol_output/SimpleAuctionEnded/states/8/reduce_combinations/graph/SimpleAuctionEnded_Mode.states does not exist
File ../res

['../results/verisol_output/Auction/states/8/reduce_combinations/graph/Auction_Mode.states',
 '../results/verisol_output/AuctionWithdraw/epa/8/reduce_combinations/graph/AuctionWithdraw_Mode.epa',
 '../results/verisol_output/AuctionWithdraw/states/8/reduce_combinations/graph/AuctionWithdraw_Mode.states',
 '../results/verisol_output/SimpleAuctionHB/epa/8/reduce_combinations/graph/SimpleAuctionHB_Mode.epa',
 '../results/verisol_output/SimpleAuctionHB/states/8/reduce_combinations/graph/SimpleAuctionHB_Mode.states',
 '../results/verisol_output/CrowdfundingT_BaseBalance/states/8/reduce_combinations/graph/CrowdfundingBase_Mode.states',
 '../results/verisol_output/SimpleAuctionEnded/epa/8/reduce_combinations/graph/SimpleAuctionEnded_Mode.epa',
 '../results/verisol_output/SimpleAuctionEnded/states/8/reduce_combinations/graph/SimpleAuctionEnded_Mode.states',
 '../results/verisol_output/AuctionEnded/epa/8/reduce_combinations/graph/AuctionEnded_Mode.epa',
 '../results/verisol_output/AuctionEnded/s

In [9]:
benchmarks = [
            "HelloBlockchain",
            "RefrigeratedTransportation",
            "HelloBlockchain_fixed",
            "RefrigeratedTransportation_fixed", 
            "RockPaperScissors",
            "CrowdfundingT_BaseBalanceFix",
            "RefundEscrowWithdraw",
]

ignore = [("CrowdfundingT_BaseBalanceFix", "s"), ("RefundEscrowWithdraw", "s")]

In [10]:
contracts_to_compare = [(c, mode) for c in benchmarks for mode in ["e", "s"] if (c, mode) not in ignore]

In [11]:
# Remove from graphs the ones that do not match with contracts_to_compare
# For example, ("CrowdfundingT_BaseBalanceFix", "s") is not in contracts_to_compare,
# but ../results/verisol_output/CrowdfundingT_BaseBalanceFix/states/8/reduce_combinations/graph/CrowdfundingBase_Mode.states is in graphs
# So we remove it from graphs

def remove_not_matching(graphs, contracts_to_compare):
    new = []
    for graph in graphs:
        for contract, mode in contracts_to_compare:
            if contract in graph and mode in graph:
                new.append(graph)
                break
    return new

graphs2 = remove_not_matching(graphs, contracts_to_compare)

files_to_discard = check_files(graphs2)
print(f"Files to discard: {files_to_discard}")

# Remove elements of 'a' from graphs2
graphs2 = [graph for graph in graphs2 if graph not in files_to_discard]

File ../results/verisol_output/CrowdfundingT_BaseBalanceFix/states/8/reduce_combinations/graph/CrowdfundingBase_Mode.states does not exist
File ../results/verisol_output/RefundEscrowWithdraw/states/8/reduce_combinations/graph/RefundEscrow_Mode.states does not exist
a: ['../results/verisol_output/CrowdfundingT_BaseBalanceFix/states/8/reduce_combinations/graph/CrowdfundingBase_Mode.states', '../results/verisol_output/RefundEscrowWithdraw/states/8/reduce_combinations/graph/RefundEscrow_Mode.states']


In [14]:
list1 = graphs2
BECA_DIR = "/Users/iangrinspan/desktop/Beca"
list2 = os.listdir(f"{BECA_DIR}/SolidityAbstractor/graph/k_8/to_600/")
list2 = [x for x in list2 if x.endswith("epa") or x.endswith("states")]
list2 = [os.path.join(f"{BECA_DIR}/SolidityAbstractor/graph/k_8/to_600/", x) for x in list2]

In [15]:
# Rename the files from list2 replacing Fixed with _fixed
def rename_files(list2):
    for file in list2:
        if "Fixed" in file:
            print(file)
            os.rename(file, file.replace("Fixed", "_fixed"))

rename_files(list2)

In [20]:
# Match each graph in graphs2 with its corresponding graph in list2 in a tuple
def match_graphs(list1, list2):
    if len(list1) != len(list2):
        print("Lists have different length")
        return None

    list1.sort()
    list2.sort()

    return list(zip(list1, list2))

graphs2 = list(map(os.path.realpath, graphs2))
res = match_graphs(graphs2, list2)

In [ ]:
res

In [22]:
# Compare each pair of graphs in res using MyGraph.equals
def compare_graphs(res):
    for graph1, graph2 in res:
        g1 = MyGraph(graph1)
        g2 = MyGraph(graph2)
        if not g1.equals(g2):
            print(f"Graphs {graph1} and {graph2} are not equal")
            print(g1.edges)
            print(g2.edges)


compare_graphs(res)